In [1]:
!pip install -q langgraph langchain langchain_ollama pydantic typing-inspect langchain-core ipython langchain_community

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langmem 0.0.4 requires httpx<0.27.0,>=0.26.0, but you have httpx 0.28.1 which is incompatible.
You should consider upgrading via the '/Users/rahulbhoyar/Projects/DataScience/GenAI-with-Python/venv/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
#!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
#!ollama serve > /dev/null 2>&1 &

In [1]:
# import time
# print("Starting Ollama service...")
# time.sleep(10)

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Starting Ollama service...
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 2049f5674b1e... 100% ▕▏ 9.0 GB                         
pulling 66b9ea09bd5b... 100% ▕▏   68 B                         
pulling eb4402837c78... 100% ▕▏ 1.5 KB                         
pulling 832dd9e0

In [ ]:
#!ollama pull qwen2.5:14b

In [2]:
import os
from dotenv import load_dotenv
_ = load_dotenv()

In [3]:
import os
os.environ["SERPAPI_API_KEY"] = "232b686fd513c39f4747dcc3811793b016b07fa7d4da0b62821a3e137bc97657"

## LangGraph

Ollama Qwen

In [4]:
import re
import json
import requests
from typing import Literal, TypedDict, List, Dict, Any, Union, Optional, Set, Annotated as AnnotatedType
from typing_extensions import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# State structure to track which agents have been used
class AgentState(TypedDict):
    messages: Annotated[List[Dict[str, Any]], "last_value"]
    next: Optional[str]
    query_parts: Dict[str, str]
    complete_agents: Set[str]
    results: Dict[str, str]
    has_math: bool
    has_search: bool

/Users/rahulbhoyar/Projects/DataScience/GenAI-with-Python/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

def get_llm_response(messages, system_prompt=None):
    # Initialize the ChatOpenAI model
    llm = ChatOpenAI(model="gpt-4", temperature=0)

    # Construct the message list for the model
    chat_messages = []
    if system_prompt:
        chat_messages.append(SystemMessage(content=system_prompt))
    chat_messages.extend(HumanMessage(content=msg['content']) for msg in messages)

    # Generate the response
    response = llm.invoke(chat_messages)
    return response.content


In [4]:
# # Initialize the LLM

# import ollama


# def get_llm_response(messages, system_prompt=None):
#     if system_prompt:
#         full_messages = [{"role": "system", "content": system_prompt}] + messages
#     else:
#         full_messages = messages

#     response = ollama.chat(
#         model="qwen2.5:14b",
#         messages=full_messages,
#         stream=False
#     )
#     return response['message']

In [6]:
def query_analyzer(state: AgentState) -> AgentState:
    messages = state["messages"]
    user_input = messages[-1]["content"]

    system_prompt = """Analyze the user query and determine:
1. If it contains a mathematical calculation
2. If it contains a search query or question
3. If it contains BOTH a calculation AND a search query

For calculations, extract just the mathematical expression.
For search queries, extract just the search question.

Respond in JSON format:
{
  "has_math": true/false,
  "has_search": true/false,
  "math_query": "extracted math expression or null",
  "search_query": "extracted search question or null"
}"""

    analysis = get_llm_response([{"role": "user", "content": user_input}], system_prompt)

    try:
        # Parse the JSON response
        analysis_json = json.loads(analysis["content"])

        new_state = state.copy()
        new_state["has_math"] = analysis_json.get("has_math", False)
        new_state["has_search"] = analysis_json.get("has_search", False)
        new_state["query_parts"] = {}
        new_state["complete_agents"] = set()
        new_state["results"] = {}

        if analysis_json.get("has_math") and analysis_json.get("math_query"):
            new_state["query_parts"]["calculator"] = analysis_json["math_query"]

        if analysis_json.get("has_search") and analysis_json.get("search_query"):
            new_state["query_parts"]["search_engine"] = analysis_json["search_query"]

        if new_state["has_math"] and new_state["has_search"]:
            # Start with calculator for composite queries
            new_state["next"] = "calculator"
        elif new_state["has_math"]:
            new_state["next"] = "calculator"
        else:
            new_state["next"] = "search_engine"

        return new_state
    except:
        # If parsing fails, use the old router logic as fallback
        return router_fallback(state)

# Fallback to the original router if analysis fails
def router_fallback(state: AgentState) -> AgentState:
    messages = state["messages"]
    user_input = messages[-1]["content"]

    system_prompt = """Determine if the user input is a mathematical calculation or a search query.
If it's a mathematical calculation, respond with "CALCULATOR".
If it's a search query or any other question, respond with "SEARCH".
Only respond with one word - either "CALCULATOR" or "SEARCH"."""

    decision = get_llm_response([{"role": "user", "content": user_input}], system_prompt)
    decision_text = decision["content"]

    new_state = state.copy()
    new_state["query_parts"] = {}
    new_state["complete_agents"] = set()
    new_state["results"] = {}

    if "CALCULATOR" in decision_text.upper():
        new_state["has_math"] = True
        new_state["has_search"] = False
        new_state["query_parts"]["calculator"] = user_input
        new_state["next"] = "calculator"
    else:
        new_state["has_math"] = False
        new_state["has_search"] = True
        new_state["query_parts"]["search_engine"] = user_input
        new_state["next"] = "search_engine"

    return new_state

def calculator(state: AgentState) -> AgentState:
    query = state["query_parts"].get("calculator", state["messages"][-1]["content"])

    system_prompt = """You are a math calculator. Solve the given mathematical expression and show your work.
When providing your answer:
1. Always include a clear statement of the final result
2. Format the final answer as: "The result is: [answer]"
3. For exponents like 2^10, calculate and show the exact value
4. Show each step of your calculation
"""

    llm_response = get_llm_response([{"role": "user", "content": query}], system_prompt)

    new_state = state.copy()
    new_state["complete_agents"].add("calculator")
    new_state["results"]["calculator"] = f"Math calculation: {query}\n\n{llm_response['content']}"

    if "search_engine" in new_state["complete_agents"] or not state["has_search"]:
        new_state["next"] = "response_combiner"
    else:
        new_state["next"] = "search_engine"

    return new_state

def search_engine(query: str) -> str:
    """Perform real web searches using SerpAPI"""
    try:
        api_key = os.environ.get("SERPAPI_API_KEY")

        if not api_key:
            return "Search error: SERPAPI_API_KEY not found in environment variables"

        # Make API request to SerpAPI
        url = "https://serpapi.com/search"
        params = {
            'q': query,
            'api_key': api_key,
            'engine': 'google',  # Use Google as the search engine
            'num': 3  # Number of results
        }

        response = requests.get(url, params=params)
        results = response.json()

        if 'error' in results:
            return f"Search error: {results['error']}"

        if 'organic_results' not in results or not results['organic_results']:
            return "No search results found"

        formatted_results = []
        for i, item in enumerate(results['organic_results'][:3], 1):
            title = item.get('title', 'No title')
            snippet = item.get('snippet', 'No description')
            link = item.get('link', 'No URL')
            formatted_results.append(
                f"{i}. {title}\n{snippet}\nSource: {link}"
            )

        return "\n\n".join(formatted_results)
    except Exception as e:
        return f"Search error: {str(e)}"

def search_engine_agent(state: AgentState) -> AgentState:
    query = state["query_parts"].get("search_engine", state["messages"][-1]["content"])

    search_result = search_engine(query)

    new_state = state.copy()
    new_state["complete_agents"].add("search_engine")
    new_state["results"]["search_engine"] = f"Search query: {query}\n\n{search_result}"

    # Check if we need to process more components or go to combiner
    if "calculator" in new_state["complete_agents"] or not state["has_math"]:
        new_state["next"] = "response_combiner"
    else:
        new_state["next"] = "calculator"

    return new_state

# Router to decide which agent should handle the query
def router(state: AgentState) -> AgentState:
    messages = state["messages"]
    user_input = messages[-1]["content"]

    system_prompt = """Determine if the user input is a mathematical calculation or a search query.
If it's a mathematical calculation, respond with "CALCULATOR".
If it's a search query or any other question, respond with "SEARCH".
Only respond with one word - either "CALCULATOR" or "SEARCH"."""

    decision = get_llm_response([{"role": "user", "content": user_input}], system_prompt)
    decision_text = decision["content"]

    if "CALCULATOR" in decision_text.upper():
        return {"messages": messages, "next": "calculator"}
    else:
        return {"messages": messages, "next": "search_engine"}

def response_combiner(state: AgentState) -> AgentState:
    system_prompt = """You are an assistant that combines results from different tools into a coherent response.
You have access to results from a calculator and/or search engine.

When both calculation and search results are available:
1. Summarize the calculation result clearly
2. Extract the most important information from the search results
3. Present both pieces of information in a clear, concise format

When only calculation results are available:
1. Clearly state the result of the calculation
2. Include the original expression and the answer

When only search results are available:
1. Extract the most relevant information from the search results
2. Provide a direct answer to the user's question when possible
3. If a specific fact is requested (like an exchange rate), state it clearly

Keep your response concise and focused on answering the user's query."""

    combiner_input = "Here are the results from different tools:\n\n"

    if "calculator" in state["results"]:
        combiner_input += state["results"]["calculator"] + "\n\n"

    if "search_engine" in state["results"]:
        combiner_input += state["results"]["search_engine"] + "\n\n"

    combiner_input += f"Original user query: {state['messages'][-1]['content']}\n"
    combiner_input += "Please combine these results into a coherent response that directly answers the user's question."

    llm_response = get_llm_response([{"role": "user", "content": combiner_input}], system_prompt)

    new_messages = state["messages"].copy()
    new_messages.append({"role": "assistant", "content": llm_response["content"]})

    return {"messages": new_messages, "next": None,
            "query_parts": state["query_parts"],
            "complete_agents": state["complete_agents"],
            "results": state["results"],
            "has_math": state["has_math"],
            "has_search": state["has_search"]}


def user_response(state: AgentState) -> AgentState:
    return {"messages": state["messages"], "next": None}


In [9]:
# Build the graph
def build_graph():
    # Define the graph
    graph = StateGraph(AgentState)

    # Add nodes
    graph.add_node("query_analyzer", query_analyzer)
    graph.add_node("calculator", calculator)
    graph.add_node("search_engine", search_engine_agent)
    graph.add_node("response_combiner", response_combiner)

    # Connect the graph
    graph.add_edge(START, "query_analyzer")

    # Add conditional edges based on analysis result
    graph.add_conditional_edges(
        "query_analyzer",
        lambda state: state["next"],
        {
            "calculator": "calculator",
            "search_engine": "search_engine"
        }
    )

    # Add conditional edges from calculator
    graph.add_conditional_edges(
        "calculator",
        lambda state: state["next"],
        {
            "search_engine": "search_engine",  # Direct link to search engine
            "response_combiner": "response_combiner"
        }
    )

    # Add conditional edges from search engine
    graph.add_conditional_edges(
        "search_engine",
        lambda state: state["next"],
        {
            "calculator": "calculator",  # Direct link to calculator
            "response_combiner": "response_combiner"
        }
    )

    graph.add_edge("response_combiner", END)

    # Compile the graph
    return graph.compile(name="LangGraph Multi-Agent System")


In [10]:
def main():
    if not os.environ.get("SERPAPI_API_KEY"):
        print("Please set the SERPAPI_API_KEY environment variable.")
        print("You can get it by signing up at https://serpapi.com/")
        return

    graph = build_graph()

    try:
        from google.colab import output
        from IPython.display import display, Markdown, HTML
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    # Process each query
    for query in example_queries:
        if IN_COLAB:
            display(HTML(f"<h2 style='background:#2c3e50; color:white; padding:10px; border-radius:5px;'>💬 Query: {query}</h2>"))
            display(Markdown(f"**👤 User:** {query}\n"))
        else:
            print("=" * 60)
            print(f"💬 Query: {query}")
            print("=" * 60)
            print("\nResults:")
            print(f"👤 User: {query}\n")

        messages = [{"role": "user", "content": query}]

        # Run the graph
        result = graph.invoke({
            "messages": messages,
            "next": None,
            "query_parts": {},
            "complete_agents": set(),
            "results": {},
            "has_math": False,
            "has_search": False
        })

        # Display tool results
        if IN_COLAB:
            display(HTML("<h3 style='background:#34495e; color:white; padding:8px; border-radius:4px;'>🛠️ Tool Results:</h3>"))
        else:
            print("🛠️ Tool Results:\n")

        if "calculator" in result["complete_agents"]:
            calc_content = result["results"]["calculator"].split("\n\n")[1]

            if IN_COLAB:
                calc_content = calc_content.replace('\\[', '$$')
                calc_content = calc_content.replace('\\]', '$$')

                display(HTML("<strong style='color:#16a085; font-size:14px;'>Using calculator:</strong>"))
                display(Markdown(calc_content))
            else:
                calc_results = calc_content.replace('\\[', '').replace('\\]', '')
                calc_results = calc_results.replace('\\boxed{', '').replace('}', '')
                print(f"Using calculator: {calc_results}\n")

        if "search_engine" in result["complete_agents"]:
            search_results = result["results"]["search_engine"].split("\n\n", 1)[1]

            if IN_COLAB:
                display(HTML("<strong style='color:#2980b9; font-size:14px;'>Using search:</strong>"))
                display(Markdown(search_results))
            else:
                print(f"Using search:\n\n{search_results}\n")

        if result["messages"][-1]["role"] == "assistant":
            response_content = result["messages"][-1]["content"]

            if IN_COLAB:
                display(HTML("<h3 style='background:#27ae60; color:white; padding:8px; border-radius:4px;'>🤖 Assistant's Response:</h3>"))

                display(HTML(f"""
                <div style='background:#f8f9fa; border-left:4px solid #27ae60; padding:15px; border-radius:4px;'>
                    <div style='color:#2c3e50; font-size:15px;'>{response_content}</div>
                </div>
                """))
            else:
                print(f"🤖 Assistant's Response: {response_content}\n")

        if IN_COLAB:
            display(HTML("<hr style='margin:30px 0; border:0; border-top:1px solid #eee;'>"))
        else:
            print("\n" + "-" * 60 + "\n")


In [11]:
example_queries = [
    "What is 2 * 4 + 90?",
    "What's the current USD to EUR exchange rate?",
    "Calculate 2^10. Also tell me about the langgraph.",
]


if __name__ == "__main__":
    main()

💬 Query: What is 2 * 4 + 90?

Results:
👤 User: What is 2 * 4 + 90?



TypeError: string indices must be integers